**Some references**

https://www.kaggle.com/code/minhsienweng/train-infer-pii-detection-deberta-v3

(no training) https://www.kaggle.com/code/manavtrivedi/0-967-nlp-sakura/notebook

In [1]:
import json
import datasets
import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer, TrainingArguments
from scipy.special import softmax
from sklearn.model_selection import train_test_split
from spacy.lang.en import English
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from ignite.metrics import Fbeta
from functools import partial
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import recall_score, precision_score
from sklearn.preprocessing import MultiLabelBinarizer
from torch.nn.functional import softmax
from peft import get_peft_model, LoraConfig

c:\Users\User\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Preparation

In [2]:
#Finding out the number of labels
data = json.load(open('data/train.json'))


all_labels = set()

for d in data:
    all_labels = all_labels.union(set(d['labels']))

print(f"{len(list(all_labels))} labels, with the following labels:\n {list(all_labels)}")
del data

label2id = {label:index for index,label in enumerate(all_labels)}
id2label = {index:label for index,label in enumerate(all_labels)}


13 labels, with the following labels:
 ['B-ID_NUM', 'I-URL_PERSONAL', 'B-USERNAME', 'I-NAME_STUDENT', 'B-STREET_ADDRESS', 'B-NAME_STUDENT', 'I-STREET_ADDRESS', 'B-PHONE_NUM', 'O', 'B-URL_PERSONAL', 'I-PHONE_NUM', 'I-ID_NUM', 'B-EMAIL']


In [3]:
print(all_labels)

{'B-ID_NUM', 'I-URL_PERSONAL', 'B-USERNAME', 'I-NAME_STUDENT', 'B-STREET_ADDRESS', 'B-NAME_STUDENT', 'I-STREET_ADDRESS', 'B-PHONE_NUM', 'O', 'B-URL_PERSONAL', 'I-PHONE_NUM', 'I-ID_NUM', 'B-EMAIL'}


In [4]:
#Change to one-hot vector
def oh_encoder(labels):  #label: array of output for each sentence

    # unique_labels = ['O', 'B-NAME_STUDENT','I-NAME_STUDENT','B-PHONE_NUM', 'I-PHONE_NUM','B-ID_NUM', 'I-ID_NUM',  'B-URL_PERSONAL','I-URL_PERSONAL',
    #                   'B-STREET_ADDRESS', 'I-STREET_ADDRESS',  'B-EMAIL', 'B-USERNAME']
    
    
    unique_labels = ['O', 'B-NAME_STUDENT','I-NAME_STUDENT','B-URL_PERSONAL', 'B-ID_NUM','I-ID_NUM','B-EMAIL','I-STREET_ADDRESS',
                     'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM','B-STREET_ADDRESS', 'I-URL_PERSONAL']
    
    labels_oh = []
    for label in labels:    #label: str
        label_oh = [float(0)]*len(unique_labels)
        for k in range(len(unique_labels)):
            if unique_labels[k] == label:
                label_oh[k] = 1
                #labels_oh.append(torch.tensor(label_oh, requires_grad=True))
                labels_oh.append(label_oh)
                break
                

    #return torch.tensor(labels_oh, requires_grad=True)
    return torch.tensor(labels_oh, requires_grad=True, dtype=float)    #list of one-hot labels as tensors



In [5]:
# #Tokenizing sentences.
# def tokenize(example, tokenizer, INFERENCE_MAX_LENGTH=3500):
#     ''' 
#     Arguments:
#     example: sentence
#     tokenizer: following DeBERTa's
#     INFERENCE_MAX_LENGTH: for truncation if needed

#     Returns:
#     dictionary of tokenized word id, with token_map, which maps characters to its initial idx
#     '''
#     text = []
#     token_map = []
#     labels = []
    
#     idx = 0
    
#     for t, label, ws in zip(example["tokens"], 
#                      example['labels'],
#                      example["trailing_whitespace"]):
        
#         text.append(t)
#         token_map.extend([idx]*len(t))
#         labels.extend([label]*len(t))
        
#         if ws:
#             text.append(" ")
#             token_map.append(-1)

#         idx += 1
    
#     text = "".join(text)
#     tokenized = tokenizer(text, return_offsets_mapping=True, truncation=False, max_length=INFERENCE_MAX_LENGTH, return_tensors='pt')

#     labels = np.array(labels)


    
#     return {
#         **tokenized,
#         "token_map": token_map,

#     }

In [6]:
def tokenize(example, tokenizer):
    import numpy as np
    # Preprocess the tokens and labels by adding trailing whitespace and labels
    tokens = []
    labels = []
    for token, label, t_ws in zip(example["tokens"], 
                                  example["labels"],
                                  example["trailing_whitespace"]):
        tokens.append(token)
        labels.extend([label] * len(token))
        # Added trailing whitespace and label if true and 
        if t_ws:
            tokens.append(" ")
            # labels.append(oh_encoder("O"))
            labels.append("O")
    
    text = "".join(tokens)
    # print(f"len(text)={len(text)}, len(tokens)={len(tokens)}")
    # tokenization without truncation
    tokenized = tokenizer(text, return_offsets_mapping=True,
                          truncation=False)
    #labels = np.array(labels)
    # Labels
    token_labels = []
    for start_idx, end_idx in tokenized.offset_mapping:
        # Added 'O' 
        if start_idx == 0 and end_idx == 0:
            #token_labels.append(label2id["O"]) 
            #token_labels.append(oh_encoder("O"))
            token_labels.append("O")
        else:
            # case when the text starts with whitespace
            if text[start_idx].isspace():
                start_idx += 1
            # Convert label to id (int)
            #label_id = label2id[labels[start_idx]]
            label_id = labels[start_idx]
            #token_labels.append(oh_encoder(label_id))
            token_labels.append(label_id)

    return {**tokenized, "labels": token_labels, "length": len(tokenized.input_ids)}

In [7]:
'''Their one hot encoding'''
# from sklearn.preprocessing import MultiLabelBinarizer
# import pandas as pd


# # Eencode labels to columns
# def encode_labels(data):
#     df = pd.DataFrame(data)
#     total = len(df)
#     df["unique_labels"] = df["labels"].apply(lambda labels: 
#                                             list(set([label.split('-')[1] for label in labels if label != 'O'])))
#     mlb = MultiLabelBinarizer()
#     #one_hot_encoded = mlb.fit_transform(df['unique_labels'])
#     one_hot_encoded = mlb.fit_transform(df['labels'])
#     one_hot_df = pd.DataFrame(one_hot_encoded, columns=mlb.classes_)
#     df = pd.concat([df, one_hot_df], axis=1)
#     # add 'POS' column that don't have 
#     df['others'] = df['unique_labels'].apply(lambda x: 1 if len(x) == 0 else 0)
#     label_classes = list(mlb.classes_) + ['others']
#     for col in label_classes:
#         subtotal = df[col].sum()
#         percent = subtotal/total * 100
#         print(f'{col}: {subtotal}  ({percent:.1f}%)')
#     return df, label_classes

# data = json.load(open('data/train.json'))
# df , label_classes = encode_labels(data)
# print(df)

'Their one hot encoding'

In [8]:
data = json.load(open('data/train.json'))
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = tokenizer = AutoTokenizer.from_pretrained(model_name)

d = data[0]
input_ids = tokenize(d, tokenizer)
tensor = torch.tensor(input_ids['input_ids']) 
tokens = tokenizer.convert_ids_to_tokens(tensor)

# print(tokens)
# print(input_ids['labels'])
# print(d['tokens'])
# print(len(tokens),len(input_ids['labels']))

# for id, label in zip(tokens, input_ids['labels']):
#     print(id, '-->', label)

# Model: DeBERTa

Using a pretrained DeBERTa, we will build a classifier head on top of it to predict the class at token level.

In [9]:
# class Classifier(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim = 13):
#         super(Classifier, self).__init__()

#         self.dropout_prob = 0.3
#         self.final_activation = nn.Softmax(dim = -1)

#         self.linear = nn.Sequential(
#             nn.ReLU(nn.Linear(input_dim, hidden_dim)),
            
#             nn.Dropout(self.dropout_prob),
#             nn.ReLU(nn.Linear(hidden_dim, hidden_dim*2)),

#             nn.Dropout(self.dropout_prob),
#             nn.ReLU(nn.Linear(hidden_dim*2, hidden_dim)),

#             nn.Linear(hidden_dim, output_dim)
#         )
       

#     def forward(self, x):
#         logit = self.linear(x)
#         return self.final_activation(logit)


In [10]:
# class Classifier(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim = 13):
#         super(Classifier, self).__init__()

#         self.dropout_prob = 0.3
#         self.final_activation = nn.Softmax(dim = -1)

#         self.dropout = nn.Dropout(self.dropout_prob)
#         self.linear1 = nn.ReLU(nn.Linear(input_dim,hidden_dim))
#         self.linear2 = nn.ReLU(nn.Linear(hidden_dim, hidden_dim*2))                            
#         self.linear3 = nn.ReLU(nn.Linear(hidden_dim*2, hidden_dim))
#         self.output = nn.Linear(hidden_dim,output_dim)

#     def forward(self, x):
#         print('now start linear1')
#         print(x.size())
#         x = self.linear1(x)

#         print('now start linear2')
#         print(x.size())
#         x = self.linear2(self.dropout(x))

#         print('now start linear3')
#         print(x.size())
#         x = self.linear3(self.dropout(x))

#         print('now start output')
#         print(x.size())
#         logit= self.output(x)
        
#         print('now start activation')
#         output =  self.final_activation(logit)
        
#         return output


In [11]:
# class Deberta_Classif(Classifier):
#     def __init__(self, model_name, classif_input, classif_hidden, classif_output = 13, finetune = False):
#         super(Deberta_Classif, self).__init__(classif_input, classif_input)
#         self.ft = finetune

#         self.extractor = AutoModelForTokenClassification.from_pretrained(model_name).base_model

#         if not finetune:
#             for param in self.extractor.parameters():
#                 param.requires_grad = False

#         self.extractor_num_param =  sum(p.numel() for p in self.extractor.parameters())
#         self.extractor_num_param_grad = sum(p.numel() for p in self.extractor.parameters() if p.requires_grad)
#         self.extractor_name = "DeBERTa"
                

#         self.classifier = Classifier(input_dim=classif_input, hidden_dim=classif_hidden ,output_dim=classif_output)
#         self.classifier_num_param = sum(p.numel() for p in self.classifier.parameters() if p.requires_grad)

        
#     def count_param(self):
        
#         if self.ft:
#             type = 'finetuned'
#             num_param = self.extractor_num_param_grad + self.classifier_num_param
#         else:
#             type = 'non-finetuned'
#             num_param = self.extractor_num_param + self.classifier_num_param

#         print(f"Number of parameters in {type} {self.extractor_name} model is {num_param:,}")
    
    
#     def forward(self, input_ids, attention_mask):
#         x = self.extractor(input_ids, attention_mask).last_hidden_state
#         x = self.classifier(x)

In [12]:

#model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
model_name = "microsoft/deberta-v3-small"
#model = Deberta_Classif(model_name, classif_input = 768, classif_hidden = 100, classif_output = 13, finetune=False)  #Extractor output has dim 768 
config = AutoModelForTokenClassification.from_pretrained(model_name).config

config.update({'num_labels': 13})

peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="FEATURE_EXTRACTION",
        inference_mode=False,
        target_modules=["query_proj", "key_proj", "value_proj", "output.dense"])

model = AutoModelForTokenClassification.from_pretrained(model_name, config = config, 
                                                        ignore_mismatched_sizes=True)
model = get_peft_model(model, peft_config)

# print(model)

for param in model.parameters():
        param.requires_grad = False

for param in model.classifier.parameters():
        param.requires_grad = True



print(f"Model's trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad): ,}")
print(f"Model's total parameters: {sum(p.numel() for p in model.parameters()): ,}")

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model's trainable parameters:  9,997
Model's total parameters:  145,148,173


In [13]:
device = 'cuda' 
model = model.to(device)
epochs = 5
tokenizer = AutoTokenizer.from_pretrained(model_name)
# collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of = 32, max_length=3500)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = epochs)
criterion = nn.CrossEntropyLoss().to(device)




c:\Users\User\anaconda3\envs\nlp\Lib\site-packages\transformers\convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
def compute_metrics(outputs, labels, unique_labels = ['O', 'B-NAME_STUDENT','I-NAME_STUDENT','B-URL_PERSONAL', 
                                                          'B-ID_NUM','I-ID_NUM','B-EMAIL','I-STREET_ADDRESS',
                                                          'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM','B-STREET_ADDRESS', 'I-URL_PERSONAL']):    
    try:
        #print("Compute metrics")
        predictions = torch.argmax(softmax(outputs, dim=2), dim=2)
        # Include prediction Remove ignored index (special tokens)
        true_preds = []
        true_labels = []
        for pred, label in zip(predictions, labels):
            true_preds.append([unique_labels[p] for p, l in zip(pred, label) if l[0] != -100])
            true_labels.append([unique_labels[torch.argmax(l)] for p, l in zip(pred, label) if l[0] != -100])
        
        mlb = MultiLabelBinarizer(classes=unique_labels)
        true_preds_bin = mlb.fit_transform(true_preds)
        true_labels_bin = mlb.transform(true_labels)
        # Compute recall, precision and f5 score
        recall = recall_score(true_labels_bin, true_preds_bin, average='samples')
        precision = precision_score(true_labels_bin, true_preds_bin, average='samples')
        # Use modified f5 score to measure the performance
        f5_score = (1 + 5*5) * (recall * precision / (5*5*precision + recall))
        result = {'f5': f5_score,  
                  'recall': recall,
                  'precision': precision}
        # print(f"result = {result}")
        return result
    except Exception as e: 
        print(e)

In [15]:
#Preparing the datasets for token classification
data = json.load(open('data/train.json'))

train_data, val_data = train_test_split(data, test_size=0.15, random_state=42)  
# train_labels = [oh_encoder(x['labels']) for x in train_data]

trainset = datasets.Dataset.from_dict({
    'full_text': [x['full_text'] for x in train_data],
    'document': [x['document'] for x in train_data],
    'tokens': [x['tokens'] for x in train_data],
    'trailing_whitespace': [x['trailing_whitespace'] for x in train_data],
    'labels' :[x['labels'] for x in train_data]
})

trainset = trainset.map(tokenize, fn_kwargs = {"tokenizer": tokenizer}, num_proc=3)
#trainset['labels'] = oh_encoder(trainset['labels'])
#train_labels = [oh_encoder(x['labels'] for x in train_data)]


# val_labels = [oh_encoder(x['labels']) for x in val_data]

valset = datasets.Dataset.from_dict({
    'full_text': [x['full_text'] for x in val_data],
    'document': [x['document'] for x in val_data],
    'tokens': [x['tokens'] for x in val_data],
    'trailing_whitespace': [x['trailing_whitespace'] for x in val_data],
    'labels' :[x['labels'] for x in val_data]
})
valset = valset.map(tokenize, fn_kwargs = {"tokenizer": tokenizer}, num_proc=3)
#valset['labels'] = oh_encoder(valset['labels'])


#val_labels = [oh_encoder(x['labels'] for x in val_data)]

del data

Map (num_proc=3): 100%|██████████| 1022/1022 [00:04<00:00, 215.17 examples/s]


In [16]:
#First item
print(f"Number of training data: {len(trainset)} || Number of validation data: {len(valset)}")

Number of training data: 5785 || Number of validation data: 1022


In [17]:
'''Not Required'''
# def to_dict(data):
#     dict_of_lists = {}
#     for d in data:
#         for key, value in d.items():
#             if key in dict_of_lists:
#                 dict_of_lists[key].append(value)
#             else:
#                 dict_of_lists[key] = [value]
#     return dict_of_lists

# trainset = to_dict(trainset)
# #trainset['labels'] = train_labels

# valset = to_dict(valset)
# #valset['labels'] = val_labels

'Not Required'

In [18]:
from torch.utils.data import Dataset, DataLoader

class Custom_data(Dataset):
    def __init__(self, data_dict):
        self.data = data_dict['input_ids']
        self.attention_mask = data_dict['attention_mask']
        self.labels = data_dict['labels']
        self.document = data_dict['document']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.attention_mask[idx]), oh_encoder(self.labels[idx]), torch.tensor(self.document[idx])

custom_train = Custom_data(trainset)
custom_val = Custom_data(valset)


In [19]:
def custom_collate(batch):
    input_ids, attention_mask, labels, doc = zip(*batch)
    # print(doc)
    # print(max(len(input_ids[0]), len(input_ids[1])), max(len(labels[0]), len(labels[1])))
    #print(input_ids[0], labels[0])

    # Pad the input_ids and labels
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    padded_attention_mask = pad_sequence(attention_mask, batch_first = True, padding_value = 0)
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # Padding for labels in token classification

    return padded_input_ids, padded_attention_mask, padded_labels, doc

batch_size = 2
train_dataloader = DataLoader(custom_train, batch_size=batch_size, collate_fn = custom_collate, shuffle = True)
val_dataloader = DataLoader(custom_val, batch_size=batch_size, collate_fn=custom_collate, shuffle=False)


'''Uncomment to check'''
# for i, (input_ids, attention_mask, labels, doc) in enumerate(val_dataloader):
#     print(f'Batch {i + 1}:')
#     print('Input IDs:', input_ids.size())
#     print('Attention_mask:', attention_mask.size())
#     print('Labels:', labels.size())
#     print('Document:', doc)
#     print()

'Uncomment to check'

In [20]:
def val(model, custom_val, batch_size, custom_collate, criterion, device):
    model.eval()

    avg_val_loss = 0
    avg_val_score = 0
    with torch.no_grad():
        val_loss = 0
        val_score = 0
        val_dataloader = DataLoader(custom_val, batch_size = batch_size, collate_fn = custom_collate, shuffle = False)

        for batch, (input_ids, attention_mask, labels, doc) in enumerate(val_dataloader):
            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask).logits
            loss = criterion(outputs, labels)
            results = compute_metrics(outputs, labels)

            val_loss += loss
            val_score += results['f5']

            if batch%200 == 0 or batch+1 == len(val_dataloader): 
                print("********** For Validation Set **********")                                                                       
                print(f"Completed {batch+1}/{len(val_dataloader)}, with current val_loss: {loss},\n current results:{results}") 

    avg_val_loss = val_loss / len(custom_val) 
    avg_val_score = val_score/len(val_dataloader)

    print(f"Average val_loss: {avg_val_loss}, avgerage val_score = {avg_val_score}")

    return avg_val_loss, avg_val_score
    


In [21]:
def train(model, custom_train, custom_val, batch_size, custom_collate, epochs, optimizer, criterion,  device):
    
    model.train()
    best_val_loss = float('inf')
    best_val_score = -float('inf')

    for epoch in range(epochs):
        avg_train_loss = 0
        avg_train_score = 0
        train_loss = 0
        train_score = 0

        
        train_dataloader = DataLoader(custom_train, batch_size = batch_size, collate_fn = custom_collate, shuffle = True)
        print('Starting training...')
        
        for batch, (input_ids, attention_mask, labels, doc) in enumerate(train_dataloader):

            optimizer.zero_grad()
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask).logits
            # print(outputs.size())
            # print(labels.size())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()


            results = compute_metrics(outputs, labels)
            train_loss += loss
            train_score+= results['f5']

            if batch%200 == 0 or batch+1 == len(train_dataloader):
                print(f"Completed {batch+1}/{len(train_dataloader)}, with current train_loss: {loss},\n current results:{results}")

        avg_train_loss = train_loss / len(custom_train)    
        avg_train_score = train_score / len(train_dataloader)

        
        print()
        print(f"Epoch {epoch+1}/{epochs}: Average train_loss = {avg_train_loss}, average train_score = {avg_train_score}")

        print()
        print("Starting to validate")
        val_loss, val_score = val(model, custom_val, batch_size, custom_collate, criterion, device)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'deberta_small_best_loss.pt')

        if val_score > best_val_score:
            best_val_score = val_score
            torch.save(model.state_dict(), 'deberta_small_best_score.pt')

        

In [22]:
batch_size = 4
epochs = 10

train(model, custom_train, custom_val, batch_size, custom_collate, epochs, optimizer, criterion, device)

Starting training...
Completed 1/1447, with current train_loss: -10861.861308290408,
 current results:{'f5': 0.5179468772433596, 'recall': 0.75, 'precision': 0.05929487179487179}
Completed 201/1447, with current train_loss: -2596369.5428390596,
 current results:{'f5': 0.7722772277227723, 'recall': 1.0, 'precision': 0.11538461538461539}
Completed 401/1447, with current train_loss: -2385374.9121821453,
 current results:{'f5': 0.6842105263157894, 'recall': 1.0, 'precision': 0.07692307692307693}
Completed 601/1447, with current train_loss: -2967851.467260467,
 current results:{'f5': 0.6842105263157894, 'recall': 1.0, 'precision': 0.07692307692307693}
Completed 801/1447, with current train_loss: -5588756.725744212,
 current results:{'f5': 0.6842105263157894, 'recall': 1.0, 'precision': 0.07692307692307693}
Completed 1001/1447, with current train_loss: -13041965.72388705,
 current results:{'f5': 0.6990782749892546, 'recall': 1.0, 'precision': 0.08202214452214453}
Completed 1201/1447, with cu

# Converting from predictions to NER labels

In [ ]:
#Used to label each token at NER stage
def find_span(target: list[str], document: list[str]) -> list[list[int]]:
    
    idx = 0
    spans = []
    span = []

    for i, token in enumerate(document):
        if token != target[idx]:
            idx = 0
            span = []
            continue
        span.append(i)
        idx += 1
        if idx == len(target):
            spans.append(span)
            span = []
            idx = 0
            continue
    
    return spans

In [ ]:
## DON'T RUN ##
#### From KAGGLE: https://www.kaggle.com/code/manavtrivedi/0-967-nlp-sakura/notebook ####

triplets = []
pairs = set()  # membership operation using set is faster O(1) than that of list O(n)
processed = []
emails = []
phone_nums = []
urls = []
streets = []

# For each prediction, token mapping, offsets, tokens, and document in the dataset
for p, token_map, offsets, tokens, doc, full_text in zip(
    preds_final, 
    ds["token_map"], 
    ds["offset_mapping"], 
    ds["tokens"], 
    ds["document"],
    ds["full_text"]
):

    # Iterate through each token prediction and its corresponding offsets
    for token_pred, (start_idx, end_idx) in zip(p, offsets):
        label_pred = id2label[str(token_pred)]  # Predicted label from token
        if start_idx + end_idx == 0:
            continue
        if token_map[start_idx] == -1:
            start_idx += 1
        while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
            start_idx += 1
        if start_idx >= len(token_map):
            break
        token_id = token_map[start_idx]  # Token ID at start index
        if label_pred in ("O", "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM") or token_id == -1:
            continue
        pair = (doc, token_id)
        if pair not in pairs:
            processed.append({"document": doc, "token": token_id, "label": label_pred, "token_str": tokens[token_id]})
            pairs.add(pair)
    
    # email
    for token_idx, token in enumerate(tokens):
        if re.fullmatch(email_regex, token) is not None:
            emails.append(
                {"document": doc, "token": token_idx, "label": "B-EMAIL", "token_str": token}
            )
                
    # phone number
    matches = phone_num_regex.findall(full_text)
    if not matches:
        continue
    for match in matches:
        target = [t.text for t in nlp.tokenizer(match)]
        matched_spans = find_span(target, tokens)
    for matched_span in matched_spans:
        for intermediate, token_idx in enumerate(matched_span):
            prefix = "I" if intermediate else "B"
            phone_nums.append(
                {"document": doc, "token": token_idx, "label": f"{prefix}-PHONE_NUM", "token_str": tokens[token_idx]}
            )
    
    # url
    matches = url_regex.findall(full_text)
    if not matches:
        continue
    for match in matches:
        target = [t.text for t in nlp.tokenizer(match)]
        matched_spans = find_span(target, tokens)
    for matched_span in matched_spans:
        for intermediate, token_idx in enumerate(matched_span):
            prefix = "I" if intermediate else "B"
            urls.append(
                {"document": doc, "token": token_idx, "label": f"{prefix}-URL_PERSONAL", "token_str": tokens[token_idx]}
            )
    
    # street
#     matches = street_regex.findall(full_text)
#     if not matches:
#         continue
#     for match in matches:
#         target = [t.text for t in nlp.tokenizer(match)]
#         matched_spans = find_span(target, tokens)
#     for matched_span in matched_spans:
#         for intermediate, token_idx in enumerate(matched_span):
#             prefix = "I" if intermediate else "B"
#             streets.append(
#                 {"document": doc, "token": token_idx, "label": f"{prefix}-STREET_ADDRESS", "token_str": tokens[token_idx]}
#             )